In [1]:
import pandas as pd

dataset = pd.read_parquet("../dataset/test_math_exercice.parquet")
dataset

,problem,level,type,solution,id,source,category
0,What is the reciprocal of $0.\overline{714285}...,Level 5,Prealgebra,Let's first convert $0.\overline{714285}$ to a...,prealgebra_1148,test,5
1,Given that $\cos a = \frac{1}{3}$ and $\cos b ...,Level 3,Precalculus,"By product-to-sum,\n\[\cos (a + b) \cos (a - b...",precalculus_732,test,6
2,"If $a$ and $b$ are integers with $a > b$, what...",Level 5,Algebra,"Let $x = \frac{a+b}{a-b}$. Then, $\frac{a-b}{a...",algebra_595,train,0
3,Two numbers are said to be 'relatively prime' ...,Level 5,Prealgebra,"Since $28=2^2\cdot 7$, a positive integer is r...",prealgebra_1419,test,5
4,How many positive integers are less than $200\...,Level 2,Prealgebra,Based on our knowledge of the decimal expansio...,prealgebra_1891,test,5
...,...,...,...,...,...,...,...
2495,Define the function $f(x) = 2x - 5$. For what ...,Level 4,Algebra,Substituting $f^{-1}(x)$ into our expression f...,algebra_1450,train,0
2496,One-half of one-seventh of $T$ equals one-thir...,Level 3,Prealgebra,"From the problem, we write the equation \[\fra...",prealgebra_495,train,5
2497,Mr. Madoff invests 1000 dollars in a fund that...,Level 4,Algebra,Let $r$ be the annual interest rate. Then aft...,algebra_1014,test,0
2498,The volume of a cylinder is 60 cubic centimete...,Level 5,Geometry,We begin by drawing a diagram: [asy]\nimport s...,geometry_582,test,2


In [2]:
import torch
from sentence_transformers import SentenceTransformer, models

model = SentenceTransformer("../models/sbert-math-problem-ContrastiveLoss-cuda/", device='cpu')
model.to('cuda')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 256, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [3]:
BATCH_SIZE = 32

dataset["embedding_vector"] = pd.Series(
        model.encode(dataset["problem"], batch_size=BATCH_SIZE, device='cuda', show_progress_bar=True).tolist()
    )

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [4]:
from datetime import datetime
dataset['created_date']= datetime.today().strftime('%Y-%m-%d')
dataset.head(3)

,problem,level,type,solution,id,source,category,embedding_vector,created_date
0,What is the reciprocal of $0.\overline{714285}...,Level 5,Prealgebra,Let's first convert $0.\overline{714285}$ to a...,prealgebra_1148,test,5,"[0.012466754764318466, 0.10739056766033173, -0...",2023-09-22
1,Given that $\cos a = \frac{1}{3}$ and $\cos b ...,Level 3,Precalculus,"By product-to-sum,\n\[\cos (a + b) \cos (a - b...",precalculus_732,test,6,"[-0.0034363302402198315, 0.08205285668373108, ...",2023-09-22
2,"If $a$ and $b$ are integers with $a > b$, what...",Level 5,Algebra,"Let $x = \frac{a+b}{a-b}$. Then, $\frac{a-b}{a...",algebra_595,train,0,"[0.009845095686614513, -0.012728335335850716, ...",2023-09-22


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   problem           2500 non-null   object  
 1   level             2500 non-null   object  
 2   type              2500 non-null   category
 3   solution          2500 non-null   object  
 4   id                2500 non-null   object  
 5   source            2500 non-null   object  
 6   category          2500 non-null   int32   
 7   embedding_vector  2500 non-null   object  
 8   created_date      2500 non-null   object  
dtypes: category(1), int32(1), object(7)
memory usage: 149.4+ KB


In [6]:
from opensearchpy import OpenSearch, RequestsHttpConnection

host = 'localhost'
port = 9200
auth = ('admin', 'admin') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
)

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\opensearchpy\connection\http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [7]:
index_name = 'math-exercice'
index_body = {
    "mappings": {
        "properties": {
            "problem": {"type": "text"},
            "level": {"type": "text"},
            "type": {"type": "text"},
            "solution": {"type": "text"},
            "id": {"type": "text"},
            "source": {"type": "text"},
            "category": {"type": "integer"},
            "created_date": {"type": "date", "format": "yyyy-MM-dd"},
            "embedding_vector": {
                "type": "knn_vector",
                "dimension": 256
            }
        }
    }
}

response = client.indices.create(index_name, body=index_body)
print('\nCreating index:')
print(response)

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(



Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'math-exercice'}


In [8]:
from pprint import pprint

documents = [
    {
            "problem": doc["problem"],
            "level": doc["level"],
            "type": doc["type"],
            "solution": doc["solution"],
            "id": doc["id"],
            "source": doc["source"],
            "category": doc["category"],
            "created_date": doc["created_date"],
            "embedding_vector": doc["embedding_vector"]
    }
    for _, doc in dataset.iterrows()
]

pprint(documents[:3])

[{'category': 5,
  'created_date': '2023-09-22',
  'embedding_vector': [0.012466754764318466,
                       0.10739056766033173,
                       -0.11800581961870193,
                       0.07028628885746002,
                       -0.09219158440828323,
                       -0.08459657430648804,
                       0.058190204203128815,
                       -0.07246341556310654,
                       0.10452862083911896,
                       -0.062575563788414,
                       -0.10683193057775497,
                       -0.03667931631207466,
                       0.013041844591498375,
                       -0.057696230709552765,
                       -0.006717165466398001,
                       0.07616379857063293,
                       -0.020801374688744545,
                       -0.022260719910264015,
                       -0.020964408293366432,
                       0.008250716142356396,
                       -0.008645243011415005,
      

In [9]:
def gendata(documents, index_name):
    for doc in documents:
        yield {
            "_op_type": "index",
            "_index": index_name,
            "_id": doc["id"],
            "_source": doc,
        }

In [10]:
documents_iterator = gendata(documents, index_name)

In [11]:
import logging
from opensearchpy import helpers

STREAM_BULK_KWARGS = {
    "max_retries": 3,
    "initial_backoff": 5,
    "max_backoff": 30,
    "raise_on_error": True,
    "raise_on_exception": True,
}

def os_streaming_bulk(client, documents_iterator):
    success, failed = 0, 0
    # list of errors to be collected is not stats_only
    errors = []
    for ok, item in helpers.streaming_bulk(client, documents_iterator, **STREAM_BULK_KWARGS):
        if not ok:
            errors.append(item)
            failed += 1
            logging.info("Opensearch streaming bulk failed")
        else:
            success += 1
    return {"success": success, "failed": failed, "errors": errors}

In [12]:
os_streaming_bulk(client, documents_iterator)

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\U

{'success': 2500, 'failed': 0, 'errors': []}

In [13]:
body = {
    "size": 3,
    "query": {
        "match_all": {}
    }
}
res = client.search(body=body, index=index_name)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'prealgebra_1148',
                    '_index': 'math-exercice',
                    '_score': 1.0,
                    '_source': {'category': 5,
                                'created_date': '2023-09-22',
                                'embedding_vector': [0.012466754764318466,
                                                     0.10739056766033173,
                                                     -0.11800581961870193,
                                                     0.07028628885746002,
                                                     -0.09219158440828323,
                                                     -0.08459657430648804,
                                                     0.058190204203128815,
                                                     -0.07246341556310654,
                                                     0.10452862083911896,
                     

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
def similar_documents_retrieval(query, top_k=10, client=client, model=model):
    query_embedding = model.encode(query)

    body = {
        "size": top_k,
        "_source": ["id", "category", "level", "type", "problem", "solution", "date"],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "knn_score",
                    "lang": "knn",
                    "params": {
                        "field": "embedding_vector",
                        "query_value": query_embedding.tolist(),
                        "space_type": "cosinesimil"
                    }
                }
            }
        }
    }

    results = client.search(body=body, index=index_name)
    results = results['hits']['hits']
    results = [{"score": r["_score"], **r["_source"]} for r in results]
    
    return results

In [15]:
def similar_documents_retrieval(query, top_k=10, client=client, model=model):
    query_embedding = model.encode(query)

    body = {
        "size": top_k,
        "_source": ["id", "category", "level", "type", "problem", "solution", "date"],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "knn_score",
                    "lang": "knn",
                    "params": {
                        "field": "embedding_vector",
                        "query_value": query_embedding.tolist(),
                        "space_type": "cosinesimil"
                    }
                }
            }
        }
    }

    results = client.search(body=body, index=index_name)
    results = results['hits']['hits']
    results = [{"score": r["_score"], **r["_source"]} for r in results]
    
    return results

In [16]:
from IPython.display import display, Latex

results = similar_documents_retrieval(query="exercice with circles and volumes")

for i, result in enumerate(results):
    display(Latex('-'*20+'METADATA'+'-'*20))
    print(i, result["score"])
    display(Latex("Type: "+ result['type']))
    display(Latex("Level: " + result['level']))
    display(Latex('-'*20+'EXERCICE'+'-'*20))
    display(Latex(result['problem']))
    display(Latex('-'*20+'SOLUTION'+'-'*20))
    display(Latex(result['solution']))
    display(Latex('='*100))
    print("\n")

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<IPython.core.display.Latex object>

0 1.8379517


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [17]:
results = similar_documents_retrieval(query="exercice on probability")

for i, result in enumerate(results):
    display(Latex('-'*20+'METADATA'+'-'*20))
    print(i, result["score"])
    display(Latex("Type: "+ result['type']))
    display(Latex("Level: " + result['level']))
    display(Latex('-'*20+'EXERCICE'+'-'*20))
    display(Latex(result['problem']))
    display(Latex('-'*20+'SOLUTION'+'-'*20))
    display(Latex(result['solution']))
    display(Latex('='*100))
    print("\n")

C:\Users\Abdurrahman_Shahid\anaconda3\envs\stage\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<IPython.core.display.Latex object>

0 1.9636564


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [18]:
#recommend similar exercices
exercice = dataset.loc[dataset["id"]=="intermediate_algebra_1586"].iloc[0]["problem"]
display(Latex(exercice))

IndexError: single positional indexer is out-of-bounds

In [19]:
results = similar_documents_retrieval(query=exercice)
for i, result in enumerate(results):
    display(Latex('-'*20+'METADATA'+'-'*20))
    print(i, result["score"])
    display(Latex("Type: "+ result['type']))
    display(Latex("Level: " + result['level']))
    display(Latex('-'*20+'EXERCICE'+'-'*20))
    display(Latex(result['problem']))
    display(Latex('-'*20+'SOLUTION'+'-'*20))
    display(Latex(result['solution']))
    display(Latex('='*100))
    print("\n")

NameError: name 'exercice' is not defined

In [20]:
from IPython.display import display, Latex
#recommend similar exercices
idx=0
print(dataset.iloc[idx][["category", "type"]])
exercice = dataset.iloc[idx]["problem"]
display(Latex(exercice))

category             5
type        Prealgebra
Name: 0, dtype: object


<IPython.core.display.Latex object>

In [23]:
results = similar_documents_retrieval(query=exercice)
for i, result in enumerate(results):
    display(Latex('-'*20+'METADATA'+'-'*20))
    print(i, result["score"])
    display(Latex("Type: "+ result['type']))
    display(Latex("Level: " + result['level']))
    display(Latex('-'*20+'EXERCICE'+'-'*20))
    display(Latex(result['problem']))
    display(Latex('-'*20+'SOLUTION'+'-'*20))
    display(Latex(result['solution']))
    display(Latex('='*100))
    print("\n")

C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<IPython.core.display.Latex object>

0 2.0


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [19]:
for _, exercice in dataset.iloc[:10].iterrows():
    print(exercice["category"])
    results = similar_documents_retrieval(exercice["problem"])
    print([r['category'] for r in results])

0
[0, 4, 4, 4, 4, 4, 4, 4, 4, 4]
5
[5, 1, 1, 1, 5, 5, 1, 1, 1, 1]
3
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
3


C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-pa

[3, 3, 3, 3, 3, 3, 3, 0, 3, 3]
5
[5, 0, 5, 0, 5, 1, 0, 3, 0, 0]
3
[3, 3, 4, 3, 3, 3, 3, 3, 3, 3]
6


C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[6, 1, 1, 5, 1, 2, 1, 1, 2, 1]
4
[4, 1, 1, 1, 1, 1, 4, 5, 5, 5]
5
[5, 2, 2, 2, 0, 2, 0, 5, 2, 2]
6
[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]


C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\musta\anaconda3\envs\work\lib\site-pa

In [8]:
response = client.indices.delete(
    index = index_name
)

print('\nDeleting index:')
print(response)


Deleting index:
{'acknowledged': True}


C:\Users\musta\anaconda3\envs\work\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
